In [ ]:
import pandas as pd
# df1 = pd.read_csv("./data/0_50.csv")
# df2 = pd.read_csv("./data/50_100.csv")
df = pd.read_csv("./data/500_600_Qverified.csv")
df = df.drop(["Unnamed: 0"],axis=1)
df = df.drop_duplicates(subset="Question")
df.to_csv("./data/0_100.csv",index=False)

In [5]:
df1 = pd.read_csv("./data/0_600_Qverified.csv")
df2 = pd.read_csv("./data/600_700_Qverified.csv")
new = pd.concat([df1,df2])
new.to_csv("./data/0_700_Qverified.csv",index=False)

In [36]:
filtered_rows = df[~df['Question'].isin(df1['Question'])]
for i in filtered_rows.Question:
    print(i)

Tổng diện tích tối thiểu của kho bảo quản dược liệu phải là bao nhiêu m2?
Để được cấp giấy phép hoạt động đo đạc và bản đồ, tổ chức trong nước cần đáp ứng điều kiện gì?
Thời gian để kiểm tra và trả lời về tính đầy đủ và hợp lệ của hồ sơ là bao lâu?
Yêu cầu đối với chương trình và đề cương của khóa đào tạo chuyên gia đánh giá hệ thống quản lý hoặc chuyên gia đánh giá chứng nhận sản phẩm bao gồm những gì?
Các yêu cầu về chương trình đào tạo cho trường trung cấp, trung tâm giáo dục nghề nghiệp có vốn đầu tư nước ngoài hoạt động không vì lợi nhuận là gì?
Thời gian để Cơ quan điều tra phòng vệ thương mại thông báo cho tổ chức, cá nhân về tính đầy đủ và hợp lệ của hồ sơ khai báo nhập khẩu là bao lâu?
Các trường hợp doanh nghiệp cho thuê lại lao động được rút tiền ký quỹ bao gồm những gì?
Doanh nghiệp được rút tiền ký quỹ trong trường hợp nào?
Thời gian để ngân hàng nhận ký quỹ thực hiện rút tiền ký quỹ là bao lâu?


In [1]:
import pandas as pd
import os
import time
from langchain.chains import RetrievalQA
import rag
from rag_init import custom_prompt_testset
from langchain_openai import ChatOpenAI
import pickle
from langchain_community.retrievers.bm25 import BM25Retriever
class CreateDataset:
    def __init__(self, csv_path, rag_class, custom_prompt_testset,fewshot_retriever,fewshot_check=False):
        self.df = pd.read_csv(csv_path)
        # self.df['Answer'] = [ast.literal_eval(i) for i in self.df['Answer']]
        # self.df['Answer'] = [i[0] for i in self.df['Answer']]
        # self.df = self.df.explode('Answer').reset_index(drop=True)
        self.questions_pre = self.df['Question'].tolist()
        self.ground_truths_pre = self.df['Answers'].tolist()
        self.questions = []
        self.ground_truths = []
        self.answers = []
        self.contexts = []
        self.fewshot_check = fewshot_check
        self.fewshot_retriever = fewshot_retriever
        # self.llm = ChatCohere(
        #     model="command-r-plus",
        #     temperature=0.1,
        #     max_tokens=None,
        #     timeout=None,
        #     cohere_api_key=self.cohere_api[self.api_idx]
        # )
        self.prompt = custom_prompt_testset
        # self.llm = ChatOpenAI(
        #     model="hf:meta-llama/Meta-Llama-3.1-70B-Instruct",    
        #     openai_api_base="https://glhf.chat/api/openai/v1",
        #     openai_api_key="glhf_7cacb0cf53d5065f1b46d6fcc98dd784",
        # )
        self.llm = ChatOpenAI(
            model="Meta-Llama-3_1-70B-Instruct",    
            openai_api_base="https://llama-3-1-70b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1",
            openai_api_key="eyJhbGciOiJFZERTQSJ9.eyJwcm9qZWN0IjoiOTU0ZDdjMGNhNWI5NDAxMzgwZDhkOGEyZDg3YmJhYmUiLCJhdWQiOiIzNzM4NjExNjY0MDQzMDM0IiwiZXhwIjoxNzcwNjM1ODk0LCJqdGkiOiJmNjkxZGI4ZS05MmYwLTRiNTktYjFjNC00MzI0Nzg0MjBhNTciLCJpc3MiOiJ0cmFpbmluZy5haS5jbG91ZC5vdmgubmV0Iiwic3ViIjoibmcxMjkwNDgtb3ZoIiwib3ZoVG9rZW4iOiJGX1RMUnp0bG5MYmIyanpsZWVremhPVjNQcVA1UFJPaktQVDQzT1pyMG1xZTJUUTdHcE9HaXNPVnBnRFhIU2NUNm96VU95QkcxdlpOUzR0VWpoTVA5UFc0UlhrcVJzeVlrZ0F6Yzh1TE9vRlk5S0Q3dE1rYW1RN0taZXZsUk5nRXBRZjdoVURWeUZMaVlMcTJ6Z3RfWkQwZmVsUHNDOEw5c2JyYUNtSGQzbExKdnA5WWM3X0VpZ0tCeG9fNGhObWpsN3lWUWt2UTJwcHNJUk1BWUpuYmtkamFXN19CdldFU19sSE5xVy1OMTJZNnpXU1VVM0FwcDBWNmcxODRaRG9SS0JmR2ZvbGxWdjRscTIzb043WlA1Y0Zmd0xYUk5LTm44SGJqcGlPVGhpTEtGY05rNzc1UGlnbWdxb1h2ZHhkeC1iQXVfMG1Xa0U0VE1jWG1wZmsyOGlReUEwVWdxQ2RHaHJTOEt4bWlrbFJZQzVJMWljXy1qU2hfSnFZanROWFM5VVlucmlfbHQ4Qy1OVjI5R1RGOFNzRFYtVHY5MXh3aTNsb3p4Y2Vsei12MEtJTXdmVHJscU4ta0ZSd1ZhLVM5M1dXLWJub3hlZjNUMU5YQ0ktRVVIeGJjMk8xZk1TQ0RUUXRZTW5oM2JuUVpUZ3JlTmg0Q0Q2dU5lb2YwRlh5VjhzLUNqdkk5R3RFaTBZSUFnd2RTNzUxNGJyMmozQTlVcDRGTkdFcXVuV0lmUk4zbFRla082clY1UEhLRDczQVEtdkRqVF9haDdyX1BuTjZTd3FraUZhazdtSld1MmRHOXJ4VWxIRmFNalVyeE1qQm1rZG9MMEpDSVlaNkExTC0xOEVDMS1WNU9xeFZFNHY2WFkxREVHTXR1SmVvdmEySmJDVk5uUHZxNm5EZ1I3Mm9HMzkyT1hYaGxXVGlTcU9NX2VDcVQyNEY5V0k3RGpXM1E2VS10Sm9PeUV0WGJERFJLRzVHQUdMVTFlQS1FaDVzSUttem1iRlhkcHNMaWUxaEswblA2VFA2bDZIa3dDME42MklJaVRCeGlNbDItdmtNeHJvbDFRVHp4TzhIbG9raWlkV3RSTjZTckttRnE5ZU8yLTU2RW5uUHVxSnBBNG5FVi1DWUdMLXFCRS1oQjUyNXJ5SU1vSmZCMjZCM3E2MTZxNU42VHZzSzJ2a3p1Tm9kMjZzblRTNzRNemsxT2kzbFcya3lVNzRGaVBRVDEzZVJCdEo2ZU41dHMwSnN5MktseV95WnduSDMwSTFrbW9OM1hPV1ZoaTItbjB4YnhsclV3T3NTbHZuQWpyQ0JOVkxtX1E5cUR3UmNEM2ZKYSJ9.IWuqfMSqSoEOTTEHOxeyXKE7ZyMB_LXbVqh8h-PNJQwQhyeaIPa64Cp9GDaXhvQhZHpfBccLqWWbli--A4HKCg",
        )
        self.llm = self.llm.bind(temperature=0,seed=0)
        self.rag_class = rag_class
        
        self.stop_processing = False
    def process_data(self):
        # Loop through the questions and invoke the LLM
        for i in range(len(self.questions_pre)):
            if self.stop_processing:
                break
            retries = 0
            while retries < 6:
                try:
                    # if retries == 3:
                    #     self.api_idx += 1
                    #     if self.api_idx == len(self.cohere_api):
                    #         self.api_idx = 0
                    #     self.change_api()
                    #     retries = 0
                    print(f"{i} {self.questions_pre[i]}")
                    fewshot_prompt = ""
                    if self.fewshot_check:
                        fewshot_subset = self.fewshot_retriever.get_relevant_documents(query=self.questions_pre[i])
                        for fewshot_qa in fewshot_subset:
                            fewshot_prompt += f"""Câu hỏi: "{fewshot_qa.page_content}"\nCâu trả lời: "{fewshot_qa.metadata["answer"]}"\n\n"""
                    prompt = self.prompt.format(context="{context}",qa_fewshot=fewshot_prompt,question = "{question}")
                    qa_chain = RetrievalQA.from_chain_type(
                        llm=self.llm,
                        chain_type="stuff",
                        retriever=self.rag_class.retriever,
                        return_source_documents=True,
                        chain_type_kwargs={
                            "prompt": rag.Prompt(prompt).set_prompt(),
                        },
                    )
                    result = qa_chain.invoke(self.questions_pre[i])
                    if result['source_documents'] == []:
                        raise ValueError("Cannot retrieve context.")
                    self.questions.append(self.questions_pre[i])
                    self.ground_truths.append(self.ground_truths_pre[i])
                    self.answers.append(result['result'])
                    print(result['result'])
                    self.contexts.append([x.page_content for x in result['source_documents']])
                    break
                except Exception as e:
                    retries += 1
                    if retries < 6:
                        print(f"Error encountered: {e}. Retrying in 10 seconds... (Attempt {retries}/{10})")
                        time.sleep(10)
                    else:
                        print(f"Error encountered: {e}. Max retries reached. Stopping the process.")
                        self.stop_processing = True
                        break
            # time.sleep(3)
    def save_data(self, output_path):
        data = {
            "questions": self.questions,
            "contexts": self.contexts,
            "ground_truths": self.ground_truths,
            "answers": self.answers
        }
        df = pd.DataFrame.from_dict(data)
        df.to_csv(output_path,index=False)
        print(f"Data saved to {output_path}")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

d:\RAG4\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
fewshot_check = False
fewshot_retriever = None
if os.path.exists("fewshot_db.pkl") and os.path.getsize("fewshot_db.pkl") > 0:
    fewshot_check = True
    with open("fewshot_db.pkl", 'rb') as file:
        fewshot_db = pickle.load(file)
    fewshot_retriever = BM25Retriever.from_documents(documents=fewshot_db,k = 5)
dataset_creator = CreateDataset(csv_path="./data/0_100_Qverified.csv", 
                                rag_class=rag.RAG(), 
                                custom_prompt_testset=custom_prompt_testset,fewshot_retriever=fewshot_retriever,fewshot_check=fewshot_check)
dataset_creator.process_data()
dataset_creator.save_data(output_path="./data/0_100_batchRAGver3.csv")

0 Ai có thẩm quyền cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm?
Cơ quan thẩm quyền quản lý CITES Việt Nam có thẩm quyền cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm.
1 Ai có thể thực hiện thủ tục "Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm"?
Tổ chức, cá nhân có nhu cầu được cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm có thể thực hiện thủ tục này.
2 Hồ sơ để thực hiện thủ tục "Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm" cần những thành phần gì?
Hồ sơ để thực hiện thủ tục "Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm" cần những thành phần sau:

- 01 Bản chính Đề nghị cấp chứng chỉ theo Mẫu số 13 ban hành kèm theo Nghị định số 06/2019/NĐ-CP ngày 22/01/2019 của Chính phủ ([Mẫu số 13 đề nghị cấp ấn phẩm Chứng chỉ Cites xuất khẩu mẫu vật lưu niệm.docx](https://csdl.dichvucong.gov.vn/web/jsp/download_file.jsp?ma='3fe505382a4491d5'))
- 01 Bản chính Bản sao sổ theo dõi hoạt động theo Mẫu số 14 ban hành kèm theo Nghị định số 06/

In [ ]:
dataset_creator.df

In [6]:
dataset_creator.contexts[1]

['Tên thủ tục: Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm\nNội dung: Trình tự thực hiện (các bước thực hiện)\n\nTrình tự thực hiện:\nTổ chức, cá nhân có nhu cầu được cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm gửi 01 hồ sơ tới Cơ quan thẩm quyền quản lý CITES Việt Nam.\nTrong thời hạn 05 ngày làm việc, kể từ ngày nhận được hồ sơ hợp lệ, Cơ quan thẩm quyền quản lý CITES Việt Nam có trách nhiệm cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm cho tổ chức, cá nhân.\nTrường hợp hồ sơ không hợp lệ, trong thời hạn 03 ngày làm việc, kể từ ngày nhận được hồ sơ, Cơ quan thẩm quyền quản lý CITES Việt Nam thông báo cho tổ chức, cá nhân biết.',
 'Tên thủ tục: Cấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu niệm\nNội dung: Chi tiết thủ tục (Mã, số quyết định, tên, tên, cấp thực hiện, loại, lĩnh vực của thủ tục)\n\nChi tiết thủ tục hành chính:\nMã thủ tục:\n1.000009\nSố quyết định:\n818 /QĐ-BNN-TCLN\nTên thủ tục:\nCấp ấn phẩm Chứng chỉ CITES xuất khẩu mẫu vật lưu

In [6]:
df1 = pd.read_csv("./data/0_400_batchRAG.csv")
df2 = pd.read_csv("./data/0_400_batchRAG_1.csv")
df = pd.concat([df1,df2])
df.to_csv("./data/0_400_batchRAG.csv",index=False)

In [ ]:
dataset_creator.save_data(output_path="./data/test_fewshot_batch_100_slicedchunk_selfquerycontent_dense+compressed.csv")

In [10]:
import pandas as pd
df = pd.read_csv("./output/error_score.csv")
df[:11]

,questions,answers,ground_truths,contexts,correctness
0,Căn cứ pháp lý cho thủ tục Cấp lại giấy xác nh...,Tôi không biết trả lời câu hỏi này.,Căn cứ pháp lý cho thủ tục Cấp lại giấy xác nh...,['Tên thủ tục: Cấp lại giấy xác nhận nội dung ...,0.0
1,Đối tượng thực hiện thủ tục Cấp lại giấy xác n...,Câu trả lời không có thông tin trong ngữ cảnh.,Đối tượng thực hiện thủ tục Cấp lại giấy xác n...,['Tên thủ tục: Cấp lại giấy xác nhận nội dung ...,0.0
2,Thủ tục Cấp Giấy phép sử dụng công cụ hỗ trợ t...,Tôi không biết trả lời câu hỏi này.,Thủ tục Cấp Giấy phép sử dụng công cụ hỗ trợ t...,['Tên thủ tục: Cấp Giấy xác nhận đăng ký công ...,0.0
3,Thủ tục Cấp Giấy phép sử dụng công cụ hỗ trợ t...,Không có thông tin về lệ phí của thủ tục này t...,Thủ tục Cấp Giấy phép sử dụng công cụ hỗ trợ t...,['Tên thủ tục: Cấp Giấy phép sử dụng công cụ h...,0.0
4,"Yêu cầu, điều kiện để thực hiện thủ tục đăng k...","Người yêu cầu đăng ký nhận cha, mẹ, con (một h...","- Bên nhận và bên được nhận là cha, mẹ, con đề...",['Tên thủ tục: Thủ tục đăng ký khai sinh kết h...,0.0
5,Thời hạn giải quyết thủ tục Khai thác và sử dụ...,Không có thông tin về thời hạn giải quyết thủ ...,Thời hạn giải quyết phụ thuộc vào hình thức nộ...,['Tên thủ tục: Khai thác và sử dụng cơ sở dữ l...,0.0
6,Các đối tượng thực hiện thủ tục Cấp lại giấy p...,Tôi không biết trả lời câu hỏi này.,Các đối tượng thực hiện thủ tục Cấp lại giấy p...,['Tên thủ tục: Cấp lại giấy phép tài nguyên nư...,0.0
7,Cơ quan thực hiện và cơ quan có thẩm quyền của...,Tôi không biết trả lời câu hỏi này.,Cơ quan thực hiện thủ tục Cấp lại giấy phép tà...,['Tên thủ tục: Cấp lại giấy phép tài nguyên nư...,0.0
8,Kết quả của thủ tục này là gì?,Kết quả của thủ tục Kiểm tra công tác nghiệm t...,Kết quả của thủ tục này là Giấy cấp phép nhập ...,['Tên thủ tục: Cấp đổi Giấy chứng nhận quyền s...,0.0
9,Thành phần hồ sơ cần có bao gồm những giấy tờ gì?,Tôi không biết trả lời câu hỏi này.,Thành phần hồ sơ cần có bao gồm: Giấy tờ của n...,['Tên thủ tục: Thủ tục xóa đăng ký quốc tịch t...,0.0


In [11]:
df[:11].to_csv("./output/error_score.csv",index=False)

In [5]:
dataset_creator.contexts[2]

['Tên thủ tục: Cấp Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương\nNội dung: Yêu cầu, điều kiện thực hiện (các yêu cầu, quy định cần thực hiện)\n\nYêu cầu, điều kiện thực hiện:\nBảo đảm thực hiện theo đúng quy định tại khoản 1 và khoản 2 Điều 50 Luật Quản lý, sử dụng vũ khí, vật liệu nổ và công cụ hỗ trợ (Luật số 14/2017/QH14 ngày 20/6/2017), cụ thể:\n“1. Tổ chức, doanh nghiệp vận chuyển tiền chất thuốc nổphải bảo đảm đủ điều kiện sau đây:\na) Tổ chức, doanh nghiệp được phép sản xuất, kinh doanh hoặc sử dụng tiền chất thuốc nổ; doanh nghiệp có đăng ký ngành, nghề vận chuyển hàng hóa;\nb) Có phương tiện đủ điều kiện vận chuyển tiền chất thuốc nổ theo tiêu chuẩn, quy chuẩn kỹ thuật về an toàn trong hoạt động tiền chất thuốc nổ;bảo đảmđiều kiệnvềan toàn, phòng cháy và chữa cháy;\nc) Người quản lý, người điều khiển phương tiện, áp tải và người phục vụ có liên quan đến vận chuyển tiền chất thuốc nổphải có trình độ chuyên môn phù hợp với vị trí, chứ

In [3]:
dataset_creator.answers

['Các hình thức nộp hồ sơ cho thủ tục Điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương là:\n\n- Nộp hồ sơ trực tiếp tại trụ sở Phòng Cảnh sát quản lý hành chính về trật tự xã hội, Công an tỉnh, thành phố trực thuộc Trung ương;\n- Nộp hồ sơ trực tuyến qua Cổng Dịch vụ công quốc gia hoặc Cổng dịch vụ công Bộ Công an;\n- Nộp hồ sơ qua dịch vụ bưu chính.',
 'Để nộp hồ sơ điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương, cần thực hiện theo những phương thức sau:\n\n- Nộp hồ sơ trực tiếp tại trụ sở Phòng Cảnh sát quản lý hành chính về trật tự xã hội, Công an tỉnh, thành phố trực thuộc Trung ương;\n- Nộp hồ sơ trực tuyến qua Cổng Dịch vụ công quốc gia hoặc Cổng dịch vụ công Bộ Công an;\n- Nộp hồ sơ qua dịch vụ bưu chính.',
 'Khi muốn điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ, người dân cần nộp hồ sơ theo những cách thức sau tại Công an tỉnh, thành phố trực thuộc trung ương:\n\n- Nộ

In [1]:
import pandas as pd
df = pd.read_csv("./data/demotest.csv")

In [ ]:
df